In [3]:
from itertools import product
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import scipy.interpolate as interp
import seaborn as sns
from tqdm import tqdm

#import splat # we should remove this dependency

import spectral_binaries as sb

tqdm.pandas()
warnings.filterwarnings("ignore")

In [13]:
sb.

AttributeError: module 'spectral_binaries' has no attribute 'core'

In [11]:
# read in single templates
DATA_DIR = sb.__path__[0]+'/../data/' # add this into sb
STFILE = DATA_DIR+'single_spectra_with_synthphot.h5'
STFILE = sb.DATA_FOLDER
df = pd.read_hdf(STFILE)
print(list(df.columns))

['SOURCE_KEY', 'NAME', 'DESIGNATION', 'RA', 'DEC', 'DISCOVERY_REFERENCE', 'OPT_TYPE', 'OPT_TYPE_REF', 'NIR_TYPE', 'NIR_TYPE_REF', 'LIT_TYPE', 'LIT_TYPE_REF', 'OBJECT_TYPE', 'J_2MASS', 'J_2MASS_E', 'H_2MASS', 'H_2MASS_E', 'KS_2MASS', 'KS_2MASS_E', 'JK_EXCESS', 'COLOR_EXTREMITY', 'LUMINOSITY_CLASS', 'METALLICITY_CLASS', 'GRAVITY_CLASS_OPTICAL', 'GRAVITY_CLASS_OPTICAL_REF', 'GRAVITY_CLASS_NIR', 'GRAVITY_CLASS_NIR_REF', 'CLUSTER', 'CLUSTER_REF', 'LIBRARY', 'BINARY', 'BINARY_REF', 'SBINARY', 'SBINARY_REF', 'COMPANION_NAME', 'COMPANION_REF', 'SIMBAD_OTYPE', 'SIMBAD_NAME', 'SIMBAD_SPT', 'SIMBAD_SPT_REF', 'SIMBAD_SEP', 'PARALLAX', 'PARALLAX_E', 'PARALLEX_REF', 'DISTANCE_PHOT', 'DISTANCE_PHOT_E', 'DISTANCE', 'DISTANCE_E', 'MU', 'MU_E', 'MU_RA', 'MU_DEC', 'MU_REF', 'VTAN', 'VTAN_E', 'RV', 'RV_E', 'RV_REF', 'VSINI', 'VSINI_E', 'VSINI_REF', 'NOTE', 'SELECT', 'DATA_KEY', 'DATA_FILE', 'INSTRUMENT', 'OBSERVATION_DATE', 'OBSERVATION_TIME', 'JULIAN_DATE', 'PROGRAM_NUMBER', 'PROGRAM_PI', 'OBSERVER', 'RE

In [7]:
DATA_DIR = "data/"
FP = "spectral_templates_data_version_jul26.h5"
path = Path(DATA_DIR, FP)
df = pd.read_hdf(path)

In [ ]:
df = df.dropna(
    subset=[
        "SPEX_TYPE",
        "system_interpolated_flux",
        "system_interpolated_noise",
        "difference_spectrum",
    ]
).reset_index(drop=True)
df["spex_type"] = df["spex_type"].apply(splat.typeToNum)

flux_df = pd.DataFrame(df["system_interpolated_flux"].tolist()).add_prefix("flux_")
noise_df = pd.DataFrame(df["system_interpolated_noise"].tolist()).add_prefix("noise_")
df_new = pd.concat([df["spex_type"], flux_df, noise_df], axis=1).ffill(axis=1).reset_index(drop=True)
df_new.tail()

In [6]:
# Initial wavegrid
wavegrid = pd.Series(df["wavegrid"].iloc[0])

# For normalization
wavegrid_scale = wavegrid[wavegrid.between(1.2, 1.4)].index

# For chi-squared
range_1 = wavegrid[wavegrid.between(0.95, 1.35)].index
range_2 = wavegrid[wavegrid.between(1.45, 1.80)].index
range_3 = wavegrid[wavegrid.between(2.00, 2.35)].index

# For SNR
flux_filter = ["flux_" + str(i) for i in wavegrid[wavegrid.between(1.1, 1.3)].index]
noise_filter = ["noise_" + str(i) for i in wavegrid[wavegrid.between(1.1, 1.3)].index]

In [4]:
def interpolate_flux_wave(wave, flux):
    f = interp.interp1d(wave, flux, assume_sorted=False, fill_value=np.nan)
    return f(wavegrid)


splat.initializeStandards()
standards_spex = splat.STDS_DWARF_SPEX

interpolated_standards = {}

for k in list(splat.STDS_DWARF_SPEX.keys()):
    if "Y" not in k:
        std = splat.STDS_DWARF_SPEX[k]
        spectrum = splat.Spectrum(
            wave=wavegrid,
            flux=interpolate_flux_wave(std.wave.value, std.flux.value),
            noise=interpolate_flux_wave(std.wave, std.noise.value),
        )
        spectrum.normalize([1, 1.3])
        interpolated_standards.update({k: spectrum})

standards, standard_types = [], []

for k in list(interpolated_standards.keys()):
    standards.append(interpolated_standards[k].flux.value)
    standard_types.append(k)


def fast_classify(flux, uncertainties, fit_range=[0.9, 1.4]):
    w = np.where(np.logical_and(wavegrid >= fit_range[0], wavegrid <= fit_range[1]))

    scales, chi = [], []

    # Loop through standards
    for std in standards:
        scales.append(
            np.nansum(flux[w] * std[w] / uncertainties[w] ** 2)
            / np.nansum((std[w] ** 2) / uncertainties[w] ** 2)
        )
        chi.append(
            np.nansum(((flux[w] - std[w] * scales[-1]) ** 2) / uncertainties[w] ** 2)
        )

    return standard_types[np.argmin(chi)]


def fast_classify_df(df):
    res = df.progress_apply(
        lambda x: splat.typeToNum(
            fast_classify(x.filter(like="flux").values, x.filter(like="noise").values)
        ),
        axis=1,
    )
    df["system_type"] = res
    return df

In [5]:
primary_range = list(range(17, 28))  # M7-L7 primaries
secondary_range = list(range(31, 39))  # T1-T8 secondaries

# Filter to only include these primaries and secondaries
primary_df = df_new[df_new["spex_type"].isin(primary_range)]
secondary_df = df_new[df_new["spex_type"].isin(secondary_range)]

primary_df.head()

,spex_type,flux_0,flux_1,flux_2,flux_3,flux_4,flux_5,flux_6,flux_7,flux_8,...,noise_399,noise_400,noise_401,noise_402,noise_403,noise_404,noise_405,noise_406,noise_407,noise_408
19,17.0,1.421944e-10,1.456862e-10,1.520721e-10,1.624367e-10,1.668434e-10,1.659980e-10,1.656944e-10,1.616858e-10,1.596456e-10,...,1.532508e-12,1.504761e-12,1.669580e-12,1.609609e-12,1.600118e-12,1.592644e-12,1.441079e-12,1.461814e-12,1.474468e-12,1.608297e-12
20,17.0,4.914253e-11,5.388633e-11,5.668498e-11,5.989768e-11,6.325788e-11,6.339925e-11,6.360359e-11,6.383454e-11,6.049485e-11,...,3.486809e-13,2.454216e-13,7.365166e-14,1.472364e-13,2.684740e-13,5.069175e-13,5.916225e-13,4.531591e-13,3.624281e-13,5.547572e-13
21,17.0,8.497064e-11,8.915361e-11,9.472622e-11,1.009404e-10,1.062999e-10,1.078474e-10,1.081270e-10,1.078484e-10,1.052933e-10,...,7.421357e-13,7.555952e-13,7.164455e-13,6.834534e-13,6.760315e-13,6.783179e-13,6.756063e-13,6.635656e-13,6.580801e-13,6.529634e-13
22,17.0,1.176569e-10,1.246898e-10,1.337510e-10,1.438664e-10,1.506459e-10,1.498014e-10,1.469220e-10,1.447904e-10,1.383410e-10,...,1.053192e-12,1.096303e-12,1.145497e-12,1.117611e-12,1.083712e-12,1.053542e-12,1.019439e-12,1.002055e-12,9.801605e-13,1.000286e-12
23,17.0,1.380912e-10,1.423047e-10,1.545272e-10,1.619234e-10,1.723275e-10,1.750613e-10,1.734114e-10,1.713281e-10,1.693047e-10,...,8.716236e-13,8.648428e-13,8.629560e-13,9.195508e-13,9.376870e-13,9.234234e-13,8.832666e-13,8.503010e-13,8.334012e-13,8.182881e-13


In [6]:
def add_two_stars(star_1, star_2):
    flux_1 = star_1.filter(like="flux")
    flux_2 = star_2.filter(like="flux")
    noise_1 = star_1.filter(like="noise")
    noise_2 = star_2.filter(like="noise")
    type_1 = (
        pd.Series(star_1["spex_type"]).rename("primary_type").reset_index(drop=True)
    )
    type_2 = (
        pd.Series(star_2["spex_type"]).rename("secondary_type").reset_index(drop=True)
    )

    new_flux = (flux_1 + flux_2).to_frame().T
    new_noise = np.sqrt(noise_1**2 + noise_2**2).to_frame().T

    return pd.concat([new_flux, new_noise, type_1, type_2], axis=1)


def compute_snr(row):
    return np.nanmedian(
        row.filter(like="flux")[flux_filter].values
        / row.filter(like="noise")[noise_filter].values
    )


In [7]:
# Get each possible spectral binary from df_new, add stars together
added_stars = []

for i in tqdm(primary_range):
    for j in secondary_range:
        prim_df = df_new[df_new["spex_type"] == i]
        sec_df = df_new[df_new["spex_type"] == j]
        all_idx = list(product(list(prim_df.index), list(sec_df.index)))
        for k in all_idx:
            prim_idx, sec_idx = k
            added = add_two_stars(prim_df.loc[prim_idx], sec_df.loc[sec_idx])
            added_stars.append(added)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [01:35<00:00,  8.68s/it]


In [8]:
added_stars = pd.concat(added_stars).reset_index(drop=True)

In [9]:
added_stars = fast_classify_df(added_stars)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29295/29295 [01:16<00:00, 381.06it/s]


In [10]:
added_stars = added_stars[added_stars["primary_type"] <= added_stars["system_type"]]
added_stars = added_stars[added_stars["system_type"].isin(primary_range)]

added_stars["system_type"].value_counts().sort_index()

17.0    2282
18.0    2692
19.0    1223
20.0    1049
21.0    5232
22.0    2483
23.0     700
24.0     202
25.0    4481
26.0     794
27.0    2124
Name: system_type, dtype: int64

In [11]:
def add_noise_util(row, low, high, step, binary):
    new_noise = row.filter(like="noise") + np.random.normal(
        row.filter(like="noise"), row.filter(like="noise").abs()
    ) * np.random.choice(np.arange(low, high, step))
    new_snr = pd.Series(
        np.nanmedian(
            row.filter(like="flux")[flux_filter].values.astype(np.float64)
            / new_noise[noise_filter].values.astype(np.float64)
        )
    )

    if binary:
        res = pd.concat(
            [
                row.filter(like="flux"),
                new_noise,
                row[["primary_type", "secondary_type", "system_type"]],
                new_snr,
            ]
        )
    else:
        res = pd.concat(
            [
                row.filter(like="flux"),
                new_noise,
                pd.Series(row["system_type"]),
                new_snr,
            ]
        )
    return res


# Get objects of SNR usually < 100
def add_random_noise_low_snr(row, binary=True):
    return add_noise_util(row, 1, 6, 0.05, binary)


# Get objects of higher SNR (> 100)
def add_random_noise_high_snr(row, binary=True):
    return add_noise_util(row, 0, 0.1, 0.01, binary)


In [ ]:
prim_sys_combs = list(added_stars.groupby(["primary_type", "system_type"]).size().index)

test = []

for c in tqdm(prim_sys_combs):
    prim_type, sys_type = c
    curr_comb = added_stars[
        (added_stars["primary_type"] == prim_type)
        & (added_stars["system_type"] == sys_type)
    ]
    with_snr_low = curr_comb.apply(add_random_noise_low_snr, axis=1)
    with_snr_high = curr_comb.apply(add_random_noise_high_snr, axis=1)
    test.append(with_snr_low)
    test.append(with_snr_high)

test = pd.concat(test)
test = test.rename(columns={0: "snr"})

  6%|████████████                                                                                                                                                                                     | 2/32 [00:21<04:32,  9.09s/it]

In [13]:
def snr_map(snr):
    if 0 <= snr < 25:
        return "1"
    elif 25 <= snr < 50:
        return "2"
    elif 50 <= snr < 100:
        return "3"
    elif 100 <= snr < 150:
        return "4"
    else:
        return "5"

In [14]:
test["snr_map"] = test["snr"].apply(snr_map)

In [15]:
test = test[test["system_type"].isin(primary_range)]

In [16]:
test["group"] = test.groupby(["primary_type", "system_type", "snr_map"]).ngroup()
test.head()

,flux_0,flux_1,flux_2,flux_3,flux_4,flux_5,flux_6,flux_7,flux_8,flux_9,...,noise_405,noise_406,noise_407,noise_408,primary_type,secondary_type,system_type,snr,snr_map,group
0,1.437066e-10,1.472403e-10,1.536146e-10,1.642271e-10,1.687348e-10,1.677666e-10,1.675241e-10,1.635971e-10,1.612938e-10,1.615802e-10,...,-3.117420e-12,7.466440e-12,-3.202049e-12,8.471620e-12,17.0,31.0,17.0,11.331502,1,0
1,1.429952e-10,1.467440e-10,1.533989e-10,1.639029e-10,1.682829e-10,1.675041e-10,1.671510e-10,1.633575e-10,1.612019e-10,1.624947e-10,...,1.112341e-12,2.660564e-12,4.273931e-12,1.033208e-12,17.0,31.0,17.0,44.842377,2,1
2,1.426537e-10,1.460524e-10,1.532803e-10,1.634423e-10,1.678607e-10,1.670228e-10,1.669187e-10,1.631174e-10,1.610143e-10,1.613976e-10,...,1.019655e-11,8.521207e-12,1.014345e-11,5.857034e-12,17.0,31.0,17.0,23.804289,1,0
3,1.430458e-10,1.465090e-10,1.524026e-10,1.627338e-10,1.680917e-10,1.674288e-10,1.671140e-10,1.629021e-10,1.608532e-10,1.609811e-10,...,4.431276e-12,4.431281e-12,4.351787e-12,4.534047e-13,17.0,31.0,17.0,52.087849,3,2
4,1.428526e-10,1.470776e-10,1.530889e-10,1.632808e-10,1.675973e-10,1.672648e-10,1.669936e-10,1.634553e-10,1.613765e-10,1.614322e-10,...,9.427057e-12,1.096719e-11,1.058507e-11,-1.289446e-12,17.0,31.0,17.0,23.133754,1,0


In [17]:
group_size = 1000


def resample_group(group_df, binary=True):
    if len(group_df) >= group_size:
        return group_df.sample(group_size)
    else:
        # Not duplicating, adding Gaussian noise with different scales to rows that were randomly sampled
        with_noise = group_df.sample(group_size - len(group_df), replace=True).apply(
            lambda x: add_noise_util(x, 0.1, 2, 0.05, binary), axis=1
        )
        with_noise.columns = list(with_noise.columns[:-2]) + ["system_type", "snr"]
        with_noise["snr_map"] = with_noise["snr"].apply(snr_map)

        return pd.concat([group_df, with_noise]).reset_index(drop=True)

In [18]:
groups_resampled = test.groupby("group").progress_apply(resample_group)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [08:07<00:00,  3.19s/it]


In [19]:
groups_resampled = groups_resampled.reset_index(drop=True).drop(columns="group")

In [20]:
# Oversample singles
singles_df = df_new[df_new["spex_type"].isin(primary_range)].rename(columns={"spex_type": "system_type"})

In [ ]:
# Add noise to singles
singles_over = []

np.seterr(all="ignore")


def add_noise_singles_helper(curr_type):
    over = curr_type.apply(lambda x: add_noise_util(x, 0, 4, 0.1, False), axis=1)
    zero_cols = {0: ["system_type", "snr"]}
    over = over.rename(
        columns=lambda c: zero_cols[c].pop(0) if c in zero_cols.keys() else c
    )
    return over


def add_noise_singles(prim_type, k=3000):
    curr_type = singles_df[singles_df["system_type"] == prim_type]#.copy(deep=True)
    curr_type["snr"] = curr_type.apply(compute_snr, axis=1)
    curr_over = [curr_type]
    over = Parallel(n_jobs=2)(delayed(add_noise_singles)(curr_type) for _ in tqdm(range(k)))
    curr_over.append(over)
    singles_over.append(pd.concat(curr_over))


for j in primary_range:
    add_noise_singles(j)
singles_over = pd.concat(singles_over).reset_index(drop=True)

In [ ]:
singles_over["snr_map"] = singles_over["snr"].apply(snr_map)

In [ ]:
singles_over["group"] = singles_over.groupby(["system_type", "snr_map"]).ngroup()
singles_over.head()

In [ ]:
singles_over.shape

In [ ]:
singles_over.isna().sum().sum()

In [ ]:
# Get singles with higher SNR values
singles_over_resampled = singles_over.groupby("group").progress_apply(resample_group, binary=False)
singles_over_resampled = singles_over_resampled.reset_index(drop=True).drop(columns="group")
singles_over_resampled.tail()

# Run Baseline Model & Calculate Difference

In [ ]:
groups_resampled["binary"] = 1
singles_over_resampled["binary"] = 0

In [ ]:
binaries_baseline = groups_resampled.drop(columns=["primary_type", "secondary_type"])
singles_baseline = singles_over_resampled
df_baseline = pd.concat([binaries_baseline, singles_baseline]).sample(frac=1).reset_index(drop=True).drop(columns="snr")
df_baseline["snr_map"] = df_baseline["snr_map"].astype(int)

In [ ]:
low_snr_baseline = df_baseline[df_baseline["snr_map"] == 1].drop(columns="snr_map")

low_snr_flux = low_snr_baseline.filter(like="flux").divide(
    np.nanmax(low_snr_baseline[["flux_" + str(i) for i in wavegrid_scale]], axis=1), axis=0
)
low_snr_noise = low_snr_baseline.filter(like="noise").abs().divide(
    np.nanmax(low_snr_baseline[["flux_" + str(i) for i in wavegrid_scale]], axis=1), axis=0
)


low_snr_singles = low_snr_baseline[low_snr_baseline["binary"] == 0]
low_snr_binaries = low_snr_baseline[low_snr_baseline["binary"] == 1]

In [ ]:
single_types = low_snr_singles["system_type"].unique()
binary_types = low_snr_binaries["system_type"].unique()
types_intersection = list(set(single_types) & set(binary_types))

low_snr_singles = low_snr_singles[low_snr_singles["system_type"].isin(types_intersection)]
low_snr_binaries = low_snr_binaries[low_snr_binaries["system_type"].isin(types_intersection)]

In [ ]:
low_snr_singles["system_type"].value_counts().sort_index()

In [ ]:
# Imbalanced because we have to account for all primary-system type combinations
# Some system types occur more often than others 
low_snr_binaries["system_type"].value_counts().sort_index()

In [ ]:
splat.initializeStandards()
standards_spex = splat.STDS_DWARF_SPEX

type_standard = {}

for spectral_type in range(10, 40):
    type_standard[spectral_type] = splat.getStandard(spectral_type)

In [ ]:
df_low_snr = pd.concat([low_snr_singles, low_snr_binaries]).sample(frac=1).reset_index(drop=True)
df_low_snr_flux = df_low_snr.filter(like="flux")
df_low_snr_flux = df_low_snr_flux + np.random.normal(df_low_snr_flux, df_low_snr.filter(like="noise").abs())
df_low_snr_baseline = pd.concat([df_low_snr_flux, df_low_snr[["system_type", "binary"]]], axis=1).dropna()
df_low_snr_baseline.replace([np.inf, -np.inf], np.nan, inplace=True)
df_low_snr_baseline.dropna(inplace=True)


# Calculate difference spectra
def fast_diff(row):
    flux_vals = row.filter(like="flux")
    noise_vals = row.filter(like="noise")
    spectral_type = row.filter(like="type").values[0]
    
    sp = splat.Spectrum(flux=flux_vals, noise=noise_vals, wave=wavegrid)
    standard = type_standard[spectral_type]
    sp.normalize()
    standard.normalize()
    
    diff = sp - standard
    abs_diff = np.abs(diff.flux.value)
    return abs_diff


df_low_snr_baseline_diffs = df_low_snr_baseline.progress_apply(fast_diff, axis=1)

In [ ]:
df_low_snr_baseline_diffs_df = pd.DataFrame(
    df_low_snr_baseline_diffs.tolist(),
    columns=["diff_" + str(i) for i in range(len(df_low_snr_baseline_diffs.iloc[0]))],
)
df_low_snr_baseline_diffs_df.head()

In [ ]:
df_low_snr_baseline = pd.concat([df_low_snr_baseline, df_low_snr_baseline_diffs_df], axis=1)

In [ ]:
df_low_snr_baseline = df_low_snr_baseline.dropna()

In [ ]:
X, y = df_low_snr_baseline.drop(columns="binary"), df_low_snr_baseline["binary"]

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

train_df = pd.concat([X_train, y_train], axis=1)
singles_train = train_df[train_df["binary"] == 0]
binaries_train = train_df[train_df["binary"] == 1]
X_train_binaries, sys_type_binaries = binaries_train.drop(columns=["system_type"]), binaries_train["system_type"]
rus = RandomUnderSampler()
X_train_binaries_re, sys_type_binaries = rus.fit_resample(X_train_binaries, sys_type_binaries)
binaries_train_re = pd.concat([X_train_binaries_re, sys_type_binaries], axis=1)
train_df = pd.concat([binaries_train_re, singles_train]).sample(frac=1)
X_train, y_train = train_df.drop(columns="binary"), train_df["binary"]

In [ ]:
binaries_train_re["system_type"].value_counts()

In [ ]:
singles_train["system_type"].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
train_pred = rf.predict(X_train)
test_pred = rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, train_pred) + "\n")
print(classification_report(y_test, test_pred))

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(estimator=rf, X=X_train, y_true=y_train);

In [ ]:
plot_confusion_matrix(estimator=rf, X=X_test, y_true=y_test);